<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter5/section4_Big%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big data? 🤗 Datasets to the rescue!

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [2]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and

In [3]:
!pip install zstandard

The Pile 是一个由 EleutherAI 开发的超大规模英语语料库，旨在为语言模型提供多样化的训练数据。它包含多个子数据集（如 PubMed Abstracts，1550 万医学摘要），可以通过 Hugging Face 的 datasets 库轻松加载和处理。加载时需注意依赖（如 zstandard）和内存管理，特别适合需要大规模、多领域文本的 NLP 研究。通过文中示例，你可以快速上手加载和探索其中的子数据集，例如 PubMed Abstracts，为后续模型训练或分析奠定基础。

In [4]:
from datasets import load_dataset

# This takes a few minutes to run, so go grab a tea or coffee while you wait :)
data_files = "https://huggingface.co/datasets/qualis2006/PUBMED_title_abstracts_2020_baseline/resolve/main/PUBMED_title_abstracts_2020_baseline.jsonl.zst"
pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
pubmed_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)_title_abstracts_2020_baseline.jsonl.zst:   0%|          | 0.00/7.98G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/49 [00:00<?, ?it/s]

Dataset({
    features: ['meta', 'text'],
    num_rows: 17722096
})

In [5]:
pubmed_dataset[0]

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

### 内存映射的魔法

`datasets` 库通过**内存映射（Memory Mapping）**技术实现了高效的内存管理，避免将整个数据集加载到 RAM 中。

#### **什么是内存映射？**
内存映射是一种操作系统技术，将文件内容映射到虚拟内存中。
- 程序可以通过内存地址直接访问文件数据，而无需将整个文件读入 RAM。
- 数据按需从磁盘加载到内存（分页加载），只占用必要的内存。

#### **如何在 datasets 中实现？**
- `datasets` 将数据集视为内存映射文件，底层依赖 **Apache Arrow** 格式和 `pyarrow` 库。
- Apache Arrow 是一种高效的列式内存格式，支持快速数据访问和处理。
- 与传统方法（如 Pandas）相比，Arrow 不需要将整个数据集加载到内存，而是通过映射直接操作文件。

#### **优势**
- **低内存占用**：即使数据集有 20 GB，只需几 GB RAM 即可操作。
- **跨进程共享**：内存映射文件可以在多个进程间共享，支持并行处理（如 `Dataset.map()`），无需复制数据。
- **高性能**：数据加载和处理速度极快，通常达到几百 MB/s 至几 GB/s。

#### **对比 Pandas**
- Pandas 的经验法则是：RAM 需为数据集大小的 5-10 倍（如 20 GB 数据需 100-200 GB RAM）。
- `datasets` 通过内存映射打破这一限制，适用于大规模数据集。

In [6]:
!pip install psutil

In [7]:
import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 921.12 MB


In [8]:
print(f"Number of files in dataset : {pubmed_dataset.dataset_size}")
size_gb = pubmed_dataset.dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

Number of files in dataset : 24453015916
Dataset size (cache file) : 22.77 GB


In [9]:
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

Iterated over 17722096 examples (about 22.8 GB) in 273.0s, i.e. 0.083 GB/s


创建流式数据（Streaming Datasets）的用处主要体现在处理大规模数据集时的高效性和灵活性。以下是对其用处的详细分析，帮助你理解为什么以及在什么场景下使用流式数据是一个明智的选择：

---

### **创建流式数据的核心用处**

1. **应对超大数据集，节省存储和内存**
   - **场景**：当数据集（如 The Pile 的 825 GB 或 Common Crawl 的数 TB）远超你的硬盘或 RAM 容量时，传统加载方式（一次性将数据读入内存）会失败。
   - **用处**：流式数据通过边下载边处理的方式，无需将整个数据集存储到本地或加载到内存中。你可以逐个或按小批次访问数据，极大减少了对硬件资源的需求。
   - **例子**：加载 PubMed Abstracts（19.54 GB）或 FreeLaw（51 GB）时，流式模式只需几 MB 的缓冲区即可操作，而非占用数十 GB 的内存。

2. **实时处理与动态工作流**
   - **场景**：在机器学习训练或数据预处理中，需要对数据进行分词、清洗或特征提取等操作，但数据量太大，无法一次性处理。
   - **用处**：流式数据支持“按需处理”，通过 `IterableDataset.map()` 等方法，你可以边加载边应用变换（如分词），无需等待整个数据集加载完成。这种实时性特别适合动态调整工作流。
   - **例子**：对 PubMed Abstracts 进行分词时，`map(lambda x: tokenizer(x["text"]))` 逐条处理数据，输出即用即得。

3. **快速原型与实验**
   - **场景**：在研究或开发初期，你可能只想快速探索数据的一个子集，而不是下载和处理全部数据。
   - **用处**：流式模式允许你通过 `take()` 或 `skip()` 快速提取部分数据进行测试，节省时间和资源。比如，只取前 1000 条数据来验证模型效果。
   - **例子**：`pubmed_dataset_streamed.take(5)` 提取前 5 条样本，用于初步分析。

4. **分布式与并行处理的支持**
   - **场景**：在分布式系统中（如多台服务器或云环境），需要将数据分发到多个节点处理。
   - **用处**：流式数据可以轻松与并行处理结合，通过缓冲区管理和批处理（`batched=True`），提高效率。同时，数据无需完整复制到每个节点，降低传输成本。
   - **例子**：结合 `map(batched=True, batch_size=1000)`，分批处理数据并行加速。

5. **组合多源数据**
   - **场景**：需要从多个大型数据源（如医学文献、法律文档、网络文本）创建一个统一的训练语料库。
   - **用处**：流式数据通过 `interleave_datasets()` 可以将多个 `IterableDataset` 交替合并，构建多样化的数据集，且无需提前下载所有文件。
   - **例子**：将 PubMed Abstracts 和 FreeLaw 合并为一个流式数据集，用于训练跨领域的语言模型。

6. **适应动态数据源**
   - **场景**：数据源是动态更新的（如实时爬取的网页或日志），无法一次性获取完整数据集。
   - **用处**：流式模式天然适合处理不断生成的数据流，你可以持续迭代新数据，保持模型训练的实时性。
   - **例子**：从在线 API 或数据库流式加载日志数据进行分析。

---

### **具体用处举例**
- **训练大模型**：像 The Pile（825 GB）这样的数据集，如果用传统方式加载，可能需要 TB 级的存储和 RAM。流式加载只需少量缓冲区即可开始训练，适合资源有限的研究者。
- **多语言语料库**：文中提到的瑞士语言比例示例（德语、法语等），通过流式加载 OSCAR 子集并按比例合并，可以快速构建定制化数据集。
- **数据探索**：在处理未知数据集时，流式模式允许你先检查少量样本，决定是否值得进一步处理，而无需下载全部数据。

---

### **与内存映射的对比**
- **内存映射（Memory Mapping）**：适用于数据集能完整存储在磁盘上但不需全加载到 RAM 的场景。它依赖本地文件系统，加载速度快，但仍需足够硬盘空间。
- **流式数据**：适用于数据集甚至无法完整下载的场景（如云端超大数据），通过网络按需获取，彻底摆脱本地存储限制。
- **用处差异**：内存映射适合静态大数据，流式数据适合动态或超大规模数据。

---

### **总结**
创建流式数据的用处在于：
1. **突破硬件限制**：处理超大或动态数据集，无需大量存储和内存。
2. **提高效率**：实时处理、快速实验、并行加速。
3. **增强灵活性**：支持多源合并和动态数据流，适应多样化需求。

对于 NLP 任务（如模型训练、数据分析），流式数据特别适合资源有限或数据规模超大的情况，是现代大数据处理不可或缺的工具。你可以将其视为“边吃边做的厨师”，而不是“先备齐所有食材再开饭”的传统方式！如果有具体场景想讨论，可以进一步告诉我哦！

In [10]:
pubmed_dataset_streamed = load_dataset(
    "json", data_files=data_files, split="train", streaming=True
)

In [11]:
next(iter(pubmed_dataset_streamed))

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x["text"]))
next(iter(tokenized_dataset))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [13]:
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
next(iter(shuffled_dataset))

{'meta': {'pmid': 1675166, 'language': 'ita'},
 'text': '[Benzodiazepine withdrawal syndrome].\nBenzodiazepines (BDZ) are widely prescribed in clinical practice for many pathological conditions, because of their anxiolytic, sedative, myorelaxant and anticonvulsant properties. The effectiveness, specificity and rapidity of action, the few side effects and the virtual absence of toxicity, have contributed to the widespread use of these compounds. In the last decade, however, the attitude towards BDZ has greatly changed, due to growing awareness and concern about dependence liability, withdrawal phenomena, and long-term side effects. Withdrawal symptoms have been singled out and specified in the contest of a well-defined syndrome with foreseeable onset, duration and remission. Psychic and physical symptoms and disorders of sensory perception can be observed. These manifestations can be suppressed by resuming treatment. The symptomatic and developmental aspects of BDZ withdrawal syndrome a

In [14]:
dataset_head = pubmed_dataset_streamed.take(5)
list(dataset_head)

[{'meta': {'pmid': 1673585, 'language': 'eng'},
  'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was

In [15]:
# Skip the first 1,000 examples and include the rest in the training set
train_dataset = shuffled_dataset.skip(1000)
# Take the first 1,000 examples for the validation set
validation_dataset = shuffled_dataset.take(1000)

In [19]:
from datasets import load_dataset

law_dataset_streamed = load_dataset(
    "json",
    data_files="https://mystic.the-eye.eu/public/AI/pile_preliminary_components/FreeLaw_Opinions.jsonl.zst",
    split="train",
    streaming=True,
)
first_example = next(iter(law_dataset_streamed))
print(first_example)

FileNotFoundError: Unable to find 'https://mystic.the-eye.eu/public/AI/pile_preliminary_components/FreeLaw_Opinions.jsonl.zst'

In [18]:
law_dataset_streamed = load_dataset(
    "json",
    data_files="https://the-eye.eu/public/AI/pile_preliminary_components/FreeLaw_Opinions.jsonl.zst",
    split="train",
    streaming=True,
)
next(iter(law_dataset_streamed))

FileNotFoundError: Unable to find 'https://the-eye.eu/public/AI/pile_preliminary_components/FreeLaw_Opinions.jsonl.zst'

In [ ]:
from itertools import islice
from datasets import interleave_datasets

combined_dataset = interleave_datasets([pubmed_dataset_streamed, law_dataset_streamed])
list(islice(combined_dataset, 2))

In [ ]:
base_url = "https://the-eye.eu/public/AI/pile/"
data_files = {
    "train": [base_url + "train/" + f"{idx:02d}.jsonl.zst" for idx in range(30)],
    "validation": base_url + "val.jsonl.zst",
    "test": base_url + "test.jsonl.zst",
}
pile_dataset = load_dataset("json", data_files=data_files, streaming=True)
next(iter(pile_dataset["train"]))